In [1]:
import json
import random
import numpy as np
import numpy.random as rd
import torch
import itertools
import torch.nn as nn
import torch.optim as optim


In [2]:
with open("nice-char.json") as f:
    data = json.load(f)

chars = sorted(set(c for lead in data for c in lead)) + [" STOP "]
chids = {c:i for i,c in enumerate(chars)} 
leads = [[chids[c] for c in lead]+[chids[" STOP "]] for lead in data]
print(len(chars))

179


In [3]:
def stream(data, window=128, mix=100000):
    while True:
        random.shuffle(leads)
        test = [c for lead in data for c in lead]
        for _ in range(mix):
            i = random.randint(0, len(test)-window-1)
            yield test[i:i+window], test[i+1:i+1+window]
iterator = stream(leads)

In [12]:
def sample(net, chars, length=50, power=1.5):
    characters = []
    inp = torch.tensor([[chids[" STOP "]]])
    h   = net.create_hidden(1)
    for i in range(length):
        inp, h = net(inp, h)
        p = inp.exp() / inp.exp().sum()
        p = (p ** power) / (p ** power).sum()
        p = p[0]
        inp    = rd.choice(len(chars), p=p.detach().numpy())
        characters.append(chars[inp])
        inp    = torch.tensor([[inp]])
    return "".join(characters)

In [5]:
window   = 500
iterator = stream(leads, window)

In [6]:
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, n_layers=2, dropout=0.5):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        self.cells   = nn.LSTM(hidden_size, hidden_size, n_layers, dropout=dropout)
        self.decoder = nn.Linear(hidden_size, output_size,)

    def forward(self, inp, hidden):
        inp = self.encoder(inp)
        output, hidden = self.cells(inp, hidden)
        output = self.decoder(output.view(output.size(0) * output.size(1), output.size(2)))
        return output, hidden

    def create_hidden(self, batch_size):
        zero = lambda: torch.zeros(self.n_layers, batch_size, self.hidden_size)
        return [zero() for i in range(self.n_layers)]

In [7]:
net = LSTM(len(chars), 512, len(chars))
loss = nn.CrossEntropyLoss(reduction="none")

In [8]:
optimizer = optim.Adam(net.parameters())
hid = net.create_hidden(1)

In [10]:
loss_weights = torch.tensor(np.linspace(0, 1, window, dtype=np.float32)**0.5)
loss_weights /= loss_weights.sum()

In [14]:
for i in range(10000):
    inp, tar = next(iterator)
    inp = torch.tensor([[i] for i in inp])
    tar = torch.tensor(tar)

    optimizer.zero_grad()
    out, _ = net(inp, hid)
    cost = loss(out, tar).dot(loss_weights)
    cost.backward()
    optimizer.step()
    i % 10 or print("\r %d %.2f" %(i, float(cost)), sample(net, chars, 80), end="") 


 180 2.33 raye Lantired math Dermer Amed fist in 197 2i hith the the and and hit ther etese the

KeyboardInterrupt: 

In [15]:
torch.save(net, "net-weights.model")

/Users/eight1911/learn/lib/python3.7/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
